In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mip_cvxpy
!pip install cylp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from cvxpy.problems.objective import Minimize
import numpy as np
import cvxpy as cp
import cvxopt
from mip_cvxpy import PYTHON_MIP
from numpy import genfromtxt



#from cvxpy.settings import CPLEX

In [ ]:
ROWS = 100
COLS = 125
START_ROW = 0
START_COL = 0
Capacity_Per_Station = 50
Cost_Per_Station = 5

def visualizer(if_station_value, stations_x, stations_y, combo_matrix):
  if_station = np.array([])
  if_station = np.append(if_station, if_station_value) # appending flattens elements of if_station array making it iterable

  for i in range(len(if_station)):
    if(round(if_station[i]) == 1):
      combo_matrix[int(stations_x[i])][int(stations_y[i])] = -5 #stations will appear as -5 in matrix

  for k in range(ROWS):
    print(combo_matrix[k])

  f = open("demofile.csv", "w")
  f.write("") #clearing file
  f.close()
  
  f = open("demofile.csv", "a")
#  for k in range(ROWS):
#    for j in range(COLS):
#      f.write(str(round(combo_matrix[k,j], 2)) + " ")
  np.savetxt("demofile.csv", combo_matrix, delimiter=",")
  f.close()
  





def run_optimization(solver, capacity, cost_per_station, comboDenSev, transportation, orig_stations_x, stations_y, combo_matrix):

  demand = comboDenSev
  demand_R = comboDenSev #creating a demand array that contains same demand values but reshaped
  
  print("transportation.shape[0] = ", transportation.shape[0] )
  for i in range(transportation.shape[0] - 1):
    demand_R = np.append(demand_R, comboDenSev, axis = 0)
  
  demand_R = np.reshape(demand_R, (transportation.shape[0],transportation.shape[1]))
  print("demand_R.shape =", demand_R.shape)
  

  #Reshaping stations_x array
  stations_x = np.reshape(orig_stations_x, (len(orig_stations_x), 1))




  if_station = cp.Variable(shape=stations_x.shape, integer=True) #array with binary values denoting station presence
  allocation = cp.Variable(shape=transportation.shape) #matrix denoting what % of each fire is being suppresed by each station
 # print("if_station.shape = ", if_station.shape)
 # print("allocation.shape = ", allocation.shape)
  objective = cp.Minimize(cost_per_station*cp.sum(if_station) + cp.sum(cp.multiply(demand_R, cp.multiply(transportation, allocation))))
# objective func = minimize(cost_to_build_station * is_station_there + demand * transportation * allocation)

  constraints = [
          cp.sum(allocation, axis=0) == 1, #each fire's demand has to be satisfied
          allocation@(demand) <= capacity*if_station, #the total amount each station is allocating the fires has to be less that its capacity
          allocation >= 0,
          if_station >= 0,
          if_station <= 1,       
  ]
  problem = cp.Problem(objective, constraints)


  solver_name = solver if isinstance(solver, str) else solver.name()
  print("solving with", solver_name)
  optimal_value = problem.solve(solver=solver, verbose = True, maximumSeconds = 60)

  print("if_station = ", if_station.value)
  print("#stations =", round(np.sum(if_station.value)))
  #print("allocation = ", allocation.value)
  print("calculated score:", optimal_value)

  visualizer(if_station.value, orig_stations_x, stations_y, combo_matrix)



def main():

  stations_x = np.array([])
  stations_y = np.array([])
  fires_x = np.array([])
  fires_y = np.array([])
  comboDenSev = np.array([])

  orig_forestden_matrix = genfromtxt('/content/drive/MyDrive/Wildfire_Project_with_PK/Code/Mask_Creation/ComboSevDens/forest.csv', delimiter=',')
  orig_severity_matrix = genfromtxt('/content/drive/MyDrive/Wildfire_Project_with_PK/Code/Mask_Creation/ComboSevDens/severity.csv', delimiter=',')

  #cropped_forestden_matrix = np.array([[-1.0 for x in range(COLS)] for y in range(ROWS)]) 
  cropped_forestden_matrix = np.negative(np.ones((ROWS,COLS)))
  #cropped_severity_matrix = np.array([[-1.0 for x in range(COLS)] for y in range(ROWS)]) 
  cropped_severity_matrix = np.negative(np.ones((ROWS,COLS)))

  for i in range(ROWS):
    for j in range(COLS):
      cropped_forestden_matrix[i][j] = orig_forestden_matrix[START_ROW + i][START_COL + j]
   

  for i in range(ROWS):
    for j in range(COLS):
      cropped_severity_matrix[i][j] = orig_severity_matrix[i][j]

  # cropped_forestden_matrix = orig_forestden_matrix
  # cropped_severity_matrix = orig_severity_matrix
  
  combo_matrix = cropped_forestden_matrix + cropped_severity_matrix

  print("combo_matrix")
  for k in range(ROWS):
    print(combo_matrix[k])

  print("\n-------------------------------------\n")
  
  
  num_stations = 0
  num_fires = 0
  for i in range(ROWS):
    for j in range(COLS):
      if(cropped_forestden_matrix[i][j] == 0.2):
        stations_x = np.append(stations_x, i)
        stations_y = np.append(stations_y, j)
        num_stations += 1
      else:
        if(cropped_forestden_matrix[i][j] > 0.2):
          fires_x = np.append(fires_x, i)
          fires_y = np.append(fires_y, j)
          comboDenSev = np.append(comboDenSev, cropped_forestden_matrix[i][j] * cropped_severity_matrix[i][j])
          num_fires += 1

  print("num_fires = ", num_fires)
  print("num_stations = ", num_stations)
  #transportation = np.array([[-1 for x in range(num_fires)] for y in range(num_stations)])
  transportation = np.negative(np.ones((num_stations,num_fires)))
  
  for i in range(num_stations):
    for j in range(num_fires):
      #distance = np.sqrt(((stations_x[i] - fires_x[j]) * (stations_x[i] - fires_x[j])) + ((stations_y[i] - fires_y[j]) * (stations_y[i] - fires_y[j])))
      #transportation[i][j] = distance
      transportation[i][j] = np.sqrt((stations_x[i] - fires_x[j])**2 + ((stations_y[i] - fires_y[j])**2))

  
  comboDenSev = np.reshape(comboDenSev, (len(comboDenSev), 1))


  #run_optimization(cp.GLPK_MI, Capacity_Per_Station, Cost_Per_Station, comboDenSev, transportation, stations_x, stations_y, combo_matrix)
  #run_optimization(cp.CPLEX, Capacity_Per_Station, Cost_Per_Station, comboDenSev, transportation, stations_x, stations_y, combo_matrix)
  run_optimization(cp.CBC, Capacity_Per_Station, Cost_Per_Station, comboDenSev, transportation, stations_x, stations_y, combo_matrix)




if __name__ == "__main__":
    main()



combo_matrix
[0.   0.   0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.   0.   0.   0.   0.
 0.   0.   0.6  0.6  0.6  0.   0.   0.   0.   0.   1.4  1.4  1.4  1.4
 1.4  1.4  1.4  1.4  1.4  1.4  1.4  1.4  1.4  1.4  0.4  0.4  0.4  0.4
 0.4  0.4  0.73 0.73 0.73 0.73 0.73 0.73 0.73 0.4  0.4  0.4  0.4  0.4
 0.4  0.4  0.4  0.4  0.4  0.73 0.73 0.73 0.33 0.33 0.33 0.   0.   0.
 0.   0.   0.2  0.2  0.2  0.2  0.2  0.2  0.2  0.   0.   0.   0.4  0.4
 0.4  0.4  0.4  0.4  1.4  1.4  1.4  1.4  1.4  0.4  0.4  0.4  0.4  0.4
 0.4  0.4  0.4  1.06 0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  1.4
 1.4  0.4  0.4  0.4  0.4  0.4  1.   0.   0.   0.   0.   0.4  0.4 ]
[0.   0.8  0.8  0.8  0.8  0.8  0.8  0.8  0.   0.   0.   0.   0.6  0.6
 0.6  0.6  0.6  0.6  0.   1.   1.   0.   0.4  0.4  0.4  0.4  1.4  1.4
 1.4  1.4  1.4  1.4  1.4  1.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4
 0.4  0.4  0.73 0.73 0.4  0.4  0.4  0.73 0.4  0.4  0.4  0.4  0.4  0.4
 0.4  0.4  0.4  0.4  0.4  0.33 0.33 0.33 0.33 0.33 0.33 0.   0.   0.2
 0.2  0.2  0